In [471]:
from IPython.display import display, HTML
import mammoth 

In [488]:
import os 

In [491]:
os.path.dirname('a/b/b/c')

'a/b/b'

In [493]:
! mammoth '../app/static/assets/NewsEdits++ MTurk Instructions.docx' --output-dir=../app/static/assets/img
! mv "../app/static/assets/img/NewsEdits++ MTurk Instructions.html" "../app/static/assets/NewsEdits++ MTurk Instructions.html"
with open("../app/static/assets/NewsEdits++ MTurk Instructions.html", "r") as f:
    html_str = f.read()

In [498]:
NavigableString('Hello')

'Hello'

In [514]:
t = soup.new_tag('div')

In [515]:
t.insert(0, NavigableString("new"))

In [516]:
t

<div>new</div>

In [495]:
import html
html.unescape(html_str)

'<h2><a id="_idte41dc3sn2"></a>Clarification</h2><h3><a id="_lugij7rod07d"></a>Simplification</h3><h4><a id="_etafincirt1k"></a>Definition:</h4><p>Reduce the complexity or breadth of discussion; may remove information </p><h4><a id="_2avejicwts4z"></a>Example:</h4><p>http://localhost:5002/check_task?source=ap&doc_id=12111&v_x=0&v_y=1</p><p><img src="1.png" /></p><p>```Before the work began in early July, New York Gov. ||| Before the work began, New York Gov.```</p><h5><a id="_syt7kfizljc4"></a>Explanation:</h5><p>“In early July” is removed because July has been mentioned in the previous sentence. Hence, it is being simplified for clarification. This is not <i>Emphasizing a fact</i> because no text is added. Neither is it a <i>Tonal Edit</i> because the edit only involves deletion of texts instead of different word choices.</p><h3><a id="_m06ux5bneekm"></a>Define term</h3><h4><a id="_jro8iqba9xo2"></a>Definition:</h4><p>The author provides a definition of a term that was not defined in 

In [475]:
display(HTML(html))

In [476]:
from bs4 import BeautifulSoup as bs

soup = bs(html)
for a in soup.find_all('a'):
    a.extract()

prettyHTML = soup.prettify()

In [477]:
print(prettyHTML[:10000])

<html>
 <body>
  <h2>
   Clarification
  </h2>
  <h3>
   Simplification
  </h3>
  <h4>
   Definition:
  </h4>
  <p>
   Reduce the complexity or breadth of discussion; may remove information
  </p>
  <h4>
   Example:
  </h4>
  <p>
   http://localhost:5002/check_task?source=ap&amp;doc_id=12111&amp;v_x=0&amp;v_y=1
  </p>
  <p>
   <img src="1.png"/>
  </p>
  <p>
   ```Before the work began in early July, New York Gov. ||| Before the work began, New York Gov.```
  </p>
  <h5>
   Explanation:
  </h5>
  <p>
   “In early July” is removed because July has been mentioned in the previous sentence. Hence, it is being simplified for clarification. This is not &lt;i&gt;Emphasizing a fact&lt;/i&gt; because no text is added. Neither is it a &lt;i&gt;Tonal Edit&lt;/i&gt; because the edit only involves deletion of texts instead of different word choices.
  </p>
  <h3>
   Define term
  </h3>
  <h4>
   Definition:
  </h4>
  <p>
   The author provides a definition of a term that was not defined in the prev

In [478]:
elems = []
for x in soup.body.children:
    if not isinstance(x, Tag):
        continue

    if ('localhost' in x.get_text()):
        continue 
        
    elems.append(x)

In [480]:
from bs4 import BeautifulSoup
import re

In [481]:
class Switch():
    def __init__(self, categories=[]):
        self.switch = {}
        self.categories = categories
        for i in self.categories:
            self.switch[i] = False
    
    def toggle(self, i):
        for j in self.categories:
            self.switch[j] = False
        self.switch[i] = True
    
    def get_on(self):
        for k, v in self.switch.items():
            if v == True:
                return k
        return False
    
    def get_level_in(self):
        key = self.get_on()
        if key:
            return int(re.search('\d', key)[0])

In [482]:
import uuid

In [483]:
class InstructionsHandler():
    def __init__(self):
        self.new_soup = BeautifulSoup()
        self.img_count = 0
        self.switch = Switch(categories=['in_h2', 'in_h3', 'in_h4', 'in_h5'])
        self.elem_dict = {
            'h2_div': None,
            'h3_div': None,
            'h4_div': None,
            'h5_div': None,
            'card_body_div': None,
            'card_body_content_div': None
        }

    def _create_div_of_same_type(self, elem, class_name=None, **kwargs):
        div = self.new_soup.new_tag('div')
        if class_name is not None:
            div.attrs['class'] = class_name
        div.append(elem)
        return div

    def _create_second_level_header_card(self, elem, link_id=None):
        if link_id is None:
            link_id = str(uuid.uuid1())

        self.elem_dict['h3_div'] = self.new_soup.new_tag('div')
        self.elem_dict['h3_div'].attrs['class'] = "second-level-category card"

        card_header_div = self.new_soup.new_tag('div')
        card_header_div.attrs['class'] = "card-header"

        button_header = self.new_soup.new_tag('button')
        button_header.attrs['class'] = 'btn btn-link collapsed'
        button_header.attrs['data-bs-toggle'] = 'collapse'
        button_header.attrs['type'] = 'button'
        button_header.attrs['href'] = '#' + link_id
        button_header.attrs['aria-expanded'] = "false"

        button_header.append(elem)
        card_header_div.append(button_header)
        self.elem_dict['h3_div'].append(card_header_div)

        self.elem_dict['card_body_div'] = self.new_soup.new_tag('div')
        self.elem_dict['card_body_div'].attrs['id'] = link_id
        self.elem_dict['card_body_div'].attrs['class'] = 'collapse'
        self.elem_dict['card_body_div'].attrs['data-parent'] = '#accordion'

        self.elem_dict['card_body_content_div'] = self.new_soup.new_tag('div')
        self.elem_dict['card_body_content_div'].attrs['class'] = 'card-body'
        return

    def _handle_rollup(self, level_in, level_to_stop=2):
        for i in range(level_in, level_to_stop, -1):
            if i == 5:
                self.elem_dict['h4_div'].append(self.elem_dict['h5_div'])
            if i == 4:
                self.elem_dict['card_body_content_div'].append(self.elem_dict['h4_div'])
            if i == 3:
                self.elem_dict['card_body_div'].append(self.elem_dict['card_body_content_div'])
                self.elem_dict['h3_div'].append(self.elem_dict['card_body_div'])
                self.elem_dict['h2_div'].append(self.elem_dict['h3_div'])                
            if i == 2:
                self.new_soup.append(self.elem_dict['h2_div'])

    def _format_image(self, img_elem):
        pass
    
    def _format_example_block(self, elem):
        pass
    
    def generate(self, elems):
        #   The card output div has to be of the format:
        #     <div class="top-level-category">
        #         <h2>Clarification</h2>
        #         <div class="second-level-category card">
        #             <div class="card-header">
        #                 <button class="btn btn-link collapsed" data-bs-toggle="collapse" type="button" href="#test" aria-expanded="false" aria-controls="collapseFour">
        #                     <h3> Title </h3>
        #                 </button> </div>
        #             <div id="test" class="collapse" data-parent="#accordion">
        #                 <div class="card-body">
        #                     <div class="definition"><h4>Definition</h4><p>...</p></div>
        #                     <div class="example"><h4>Example</h4><div class="example-block"><p>...</p></div></div>
        #                     <div class="explanation"><h5>Explanation</h5><p>...</p></div>
        #                 </div>
        #             </div>
        #         </div>
        #    </div>

        for elem in elems:
            if elem.name == 'h2':
                level_in = self.switch.get_level_in()
                if level_in is not None:
                    self._handle_rollup(level_in, level_to_stop=1)
                self.elem_dict['h2_div'] = self._create_div_of_same_type(elem, 'top-level-category')
                self.switch.toggle('in_h2')

            elif elem.name == 'h3':
                level_in = self.switch.get_level_in()
                # end the card, wrap it all up
                if level_in >= 3:
                    self._handle_rollup(level_in, level_to_stop=2)
                self._create_second_level_header_card(elem)
                self.switch.toggle('in_h3')

            elif elem.name == 'h4':
                level_in = self.switch.get_level_in()
                if level_in >= 4:
                    self._handle_rollup(level_in, level_to_stop=2)
                class_name = elem.text.replace(':', '').lower()
                self.elem_dict['h4_div'] = self._create_div_of_same_type(elem, class_name)
                self.switch.toggle('in_h4')

            # child-most elements
            elif elem.name == 'h5':
                self.elem_dict['h5_div'] = self._create_div_of_same_type(elem, 'explanation')
                self.switch.toggle('in_h5')

            elif '```' in elem.get_text():
                example_block_div = self._create_div_of_same_type(elem, 'example-block')
                self.elem_dict['h4_div'].append(example_block_div)

            elif elem.find('img') is not None:
                img_div = self._create_div_of_same_type(elem, 'example-image hidden')
                self.elem_dict['h4_div'].append(img_div)

            else:
                level_in = self.switch.get_level_in()
                self.elem_dict[f'h{level_in}_div'].append(elem)

        self.new_soup.append(self.elem_dict['h2_div'])
        return self.new_soup



In [484]:
ih = InstructionsHandler()
generated_html = ih.generate(elems)

In [487]:
print(generated_html.prettify())

<div class="top-level-category">
 <h2>
  Clarification
 </h2>
 <div class="second-level-category card">
  <div class="card-header">
   <button aria-expanded="false" class="btn btn-link collapsed" data-bs-toggle="collapse" href="#4d7c11ee-6096-11ed-9cd0-acde48001122" type="button">
    <h3>
     Simplification
    </h3>
   </button>
  </div>
  <div class="collapse" data-parent="#accordion" id="4d7c11ee-6096-11ed-9cd0-acde48001122">
   <div class="card-body">
    <div class="definition">
     <h4>
      Definition:
     </h4>
     <p>
      Reduce the complexity or breadth of discussion; may remove information
     </p>
    </div>
    <div class="example">
     <h4>
      Example:
     </h4>
     <div class="example-image hidden">
      <p>
       <img src="1.png"/>
      </p>
     </div>
     <div class="example-block">
      <p>
       ```Before the work began in early July, New York Gov. ||| Before the work began, New York Gov.```
      </p>
     </div>
     <div class="explanation">


In [485]:
display(HTML(str(generated_html)))

In [466]:
print(generated_html.prettify())

<div class="top-level-category">
 <h2>
  Clarification
 </h2>
 <div class="second-level-category card">
  <div class="card-header">
   <button aria-expanded="false" class="btn btn-link collapsed" data-bs-toggle="collapse" href="#28b8d3d4-5ef1-11ed-9cd0-acde48001122" type="button">
    <h3>
     Simplification
    </h3>
   </button>
  </div>
  <div class="collapse" data-parent="#accordion" id="28b8d3d4-5ef1-11ed-9cd0-acde48001122">
   <div class="card-body">
    <div class="definition">
     <h4>
      Definition:
     </h4>
     <p>
      Reduce the complexity or breadth of discussion; may remove information
     </p>
    </div>
    <div class="example">
     <h4>
      Example:
     </h4>
     <div class="example-image hidden">
      <p>
       <img src="1.png"/>
      </p>
     </div>
     <div class="example-block">
      <p>
       ```Before the work began in early July, New York Gov. ||| Before the work began, New York Gov.```
      </p>
     </div>
     <div class="explanation">


# Create a Nested Dict that can be inpreted by Javascript

In [355]:
from collections import defaultdict
from copy import deepcopy

In [380]:
def create_dict_of_same_type(elem, class_name=None, attrs=None):
    div = {
        'type': 'div',
        'children': []
    }
    if class_name is not None:
        div['class'] = class_name
    elem_dict = {'type': elem.name, 'content': elem.get_text()}
    if attrs is not None:
        elem_dict.update(attrs)
    div['content'] = elem_dict
    return div
    
# elements
switch = Switch(categories=['in_h2', 'in_h3', 'in_h4', 'in_h5'])
output_dict = []
temp_elem_dict = defaultdict(lambda : {'children': []})
for elem in elems:
    if elem.name == 'h2':
        level_in = switch.get_level_in()
        if level_in is not None:
            for i in list(range(level_in, 2, -1)):
                temp_elem_dict[f'h{i-1}_div']['children'].append(deepcopy(temp_elem_dict[f'h{i}_div']))
            output_dict.append(deepcopy(temp_elem_dict['h2_div']))
        temp_elem_dict['h2_div'] = create_dict_of_same_type(elem, 'top-level-category')
        switch.toggle('in_h2')
    
    ### convert this into a card?
    elif elem.name == 'h3':
        level_in = switch.get_level_in()
        if level_in >= 3:
            for i in list(range(level_in, 2, -1)):
                temp_elem_dict[f'h{i-1}_div']['children'].append(deepcopy(temp_elem_dict[f'h{i}_div']))
            
        temp_elem_dict['h3_div'] = create_dict_of_same_type(elem, 'second-level-category')
        switch.toggle('in_h3')
        
    elif elem.name == 'h4':
        level_in = switch.get_level_in()
        if level_in >= 4:
            for i in list(range(level_in, 3, -1)):
                temp_elem_dict[f'h{i-1}_div']['children'].append(deepcopy(temp_elem_dict[f'h{i}_div']))

        class_name = elem.text.replace(':', '').lower()
        temp_elem_dict['h4_div'] = create_dict_of_same_type(elem, class_name)
        switch.toggle('in_h4')
    
    # child-most elements
    elif elem.name == 'h5':
        temp_elem_dict['h5_div'] = create_dict_of_same_type(elem, 'explanation')
        switch.toggle('in_h5')
        
    elif '```' in elem.get_text():
        example_block_div = create_dict_of_same_type(elem, 'example-block')
        temp_elem_dict['h4_div']['children'].append(deepcopy(example_block_div))
    
    elif elem.find('img') is not None:
        img = elem.find('img')
        img_div = create_dict_of_same_type(img, 'example-image hidden', attrs=img.attrs)
        temp_elem_dict['h4_div']['children'].append(deepcopy(img_div))
    
    else:
        level_in = switch.get_level_in()
        other_div = create_dict_of_same_type(elem)
        temp_elem_dict[f'h{level_in}_div']['children'].append(other_div)
        
output_dict.append(temp_elem_dict['h2_div'])

In [383]:
import json

In [386]:
ls ../app/assets/

img/


In [387]:
with open('../app/assets/instructions.json', 'w') as f:
    json.dump(output_dict, f)

In [390]:
output_dict[0].keys()

dict_keys(['type', 'children', 'class', 'content'])

In [394]:
output_dict[0]['children'][0]

{'type': 'div',
 'children': [{'type': 'div',
   'children': [{'type': 'div',
     'children': [],
     'content': {'type': 'p',
      'content': 'Reduce the complexity or breadth of discussion; may remove information '}}],
   'class': 'definition',
   'content': {'type': 'h4', 'content': 'Definition:'}},
  {'type': 'div',
   'children': [{'type': 'div',
     'children': [],
     'class': 'example-image hidden',
     'content': {'type': 'img', 'content': '', 'src': '1.png'}},
    {'type': 'div',
     'children': [],
     'class': 'example-block',
     'content': {'type': 'p',
      'content': '```Before the work began in early July, New York Gov. ||| Before the work began, New York Gov.```'}},
    {'type': 'div',
     'children': [{'type': 'div',
       'children': [],
       'content': {'type': 'p',
        'content': '“In early July” is removed because July has been mentioned in the previous sentence. Hence, it is being simplified for clarification.'}}],
     'class': 'explanation',


In [205]:
'```' in elems[:10][-4].get_text()

True

In [216]:
h4_div

<div class="example"><h4>Example:</h4><div class="example"><p>```The parties had met to hold last-minute talks to avoid a government collapse. |||```</p></div></div>

In [138]:
t = elems[0]

In [139]:
t.name

'h3'

In [99]:
elems[0].get_text()

'Simplification'

In [92]:
type(elems[1])

bs4.element.Tag

In [93]:
from bs4.element import Tag, NavigableString

In [77]:
e

'```'

In [61]:
soup.find_all(text='Example:')

['Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:',
 'Example:']

# Read from ODT

In [13]:
! pip install odfpy

In [16]:
from odf.opendocument import load
doc = load("/Users/alex/Downloads/NewsEdits++ MTurk Instructions.odt")

In [31]:
doc